In [10]:
import boto3
import json
import time

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
iam = boto3.client('iam')
ssm = boto3.client('ssm')

In [2]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='174.19.0.0/24')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "MySQL-DynamoDB"}])
vpc.wait_until_available()

In [3]:
# Crear routable
routetable_1 = vpc.create_route_table()
routetable_1.create_tags(Tags=[{"Key": "Name", "Value": "MySQL-DynamoDB-RT"}])

[ec2.Tag(resource_id='rtb-0746c1279820d0329', key='Name', value='MySQL-DynamoDB-RT')]

In [4]:
# Crear subredes:
subnet_1 = ec2.create_subnet(
    CidrBlock='174.19.0.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)

subnet_1.create_tags(Tags=[{"Key": "Name", "Value": "MySQL-DynamoDB-SN"}])

[ec2.Tag(resource_id='subnet-02d5d8f804ca6f4ce', key='Name', value='MySQL-DynamoDB-SN')]

In [5]:
# Asociar la tabla de ruta con las subredes:
routetable_1.associate_with_subnet(SubnetId = subnet_1.id)

ec2.RouteTableAssociation(id='rtbassoc-0f5cc125201e3036a')

In [7]:
# Create Internet Gateway
igw = ec2.create_internet_gateway()

# Attach Internet Gateway to VPC Applicance
vpc.attach_internet_gateway(InternetGatewayId=igw.id);

In [6]:
# Crear grupo de seguridad
security_group_1 = vpc.create_security_group(
    GroupName='MySQL-DynamoDB-SG',
    Description='Allow SSH access',
    VpcId=vpc.id
)

In [8]:
# Crear una Elastic IP address
elastic_ip_1 = ec2_client.allocate_address(Domain='vpc')

In [9]:
# Añada una ruta a la tabla de enrutamiento de la VPC Applicance para el internet natgateway
routetable_1.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw.id
);

ec2.Route(route_table_id='rtb-0746c1279820d0329', destination_cidr_block='0.0.0.0/0')

In [24]:
# Create a role
role_name = 'MySQL-DynamoDB-Role'
policy_document = {
  "Version": "2012-10-17",
  "Statement": [
  {
    "Effect": "Allow",
    "Principal": {
      "Service": "ec2.amazonaws.com"
    },
    "Action": "sts:AssumeRole"
  }
  ]
}

ssm_role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(policy_document)
)

In [25]:
# Attach the policy to the role
iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'
)

{'ResponseMetadata': {'RequestId': '9fdde080-a7b5-4f13-8e3a-736c805ecd89',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9fdde080-a7b5-4f13-8e3a-736c805ecd89',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 13 Mar 2023 21:35:16 GMT'},
  'RetryAttempts': 0}}

In [26]:
# Create an instance profile
instance_profile_name = 'MySQL-DynamoDB-Profile'
instance_profile = iam.create_instance_profile(
    InstanceProfileName=instance_profile_name
)

EntityAlreadyExistsException: An error occurred (EntityAlreadyExists) when calling the CreateInstanceProfile operation: Instance Profile MySQL-DynamoDB-Profile already exists.

In [27]:
# Add the role that you created to the instance profile:
iam.add_role_to_instance_profile(
    InstanceProfileName=instance_profile_name,
    RoleName=role_name
)

{'ResponseMetadata': {'RequestId': 'c2497aa9-b009-4037-9b63-1c9939e5e16d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c2497aa9-b009-4037-9b63-1c9939e5e16d',
   'content-type': 'text/xml',
   'content-length': '228',
   'date': 'Mon, 13 Mar 2023 21:35:29 GMT'},
  'RetryAttempts': 0}}

In [13]:
# Enable Dns in VPC
vpc.modify_attribute(EnableDnsSupport={'Value': True})
vpc.modify_attribute(EnableDnsHostnames={'Value': True})

{'ResponseMetadata': {'RequestId': 'b2cd69bf-92bb-4fd6-ac0c-a7055ddba116',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b2cd69bf-92bb-4fd6-ac0c-a7055ddba116',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '237',
   'date': 'Mon, 13 Mar 2023 21:20:14 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [29]:
# Create security group enable HHTPS:
security_group_https = vpc.create_security_group(
    GroupName='MySQL-DynamoDB',
    Description='Allow HTTPS access',
    VpcId=vpc.id,
)

response = ec2_client.authorize_security_group_ingress(
    GroupId=security_group_https.id,
    IpPermissions=[
        {'FromPort': 443,
            'IpProtocol': 'tcp',
            'IpRanges': [
                {
                    'CidrIp': vpc.cidr_block,
                    'Description': 'Allow HTTPS access'
                }
            ],
            'ToPort': 443
        }
    ]
)

ClientError: An error occurred (InvalidGroup.Duplicate) when calling the CreateSecurityGroup operation: The security group 'MySQL-DynamoDB' already exists for VPC 'vpc-0d7d59533d0399986'

In [28]:
# Creates a VPC endpoint for SSM:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssm',
    VpcEndpointType='Interface',
    SubnetIds=[subnet_1.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)
# Creates a VPC endpoint for EC2 messages:
vpc_endpoint_ec2msg = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ec2messages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet_1.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)
# Creates a VPC endpoint for SSMMessages:
vpc_endpoint_ssm = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.' + region_aws + '.ssmmessages',
    VpcEndpointType='Interface',
    SubnetIds=[subnet_1.id],
    PrivateDnsEnabled=True,
    SecurityGroupIds=[security_group_https.id]
)

ClientError: An error occurred (InvalidParameter) when calling the CreateVpcEndpoint operation: private-dns-enabled cannot be set because there is already a conflicting DNS domain for ssm.us-east-1.amazonaws.com in the VPC vpc-0d7d59533d0399986

In [16]:
def vpc_endpoint_status(vpc_endpoint_id):
    vpc_endpoint = ec2_client.describe_vpc_endpoints(
        VpcEndpointIds=[vpc_endpoint_id]
    )
    return vpc_endpoint['VpcEndpoints'][0]['State']

print('Waiting for VPC endpoints to be available...')
while vpc_endpoint_status(vpc_endpoint_ssm['VpcEndpoint']['VpcEndpointId']) != 'available':
    time.sleep(20) 
print('\nSSM endpoint is available!')

Waiting for VPC endpoints to be available...

SSM endpoint is available!


In [17]:
# Consulte en SSM el último ID de AMI de Amazon Linux 2 disponible en su región y guárdelo como variable de entorno:
ssm_response = ssm.get_parameter(
    Name='/aws/service/ami-amazon-linux-latest/amzn2-ami-hvm-x86_64-gp2'
)

In [18]:
# Create a key pair
key_name = 'MySQL-DynamoDB-KeyPair'
keypair = ec2.create_key_pair(KeyName=key_name)

In [30]:
# Create a EC2 instance:
instance_1 = ec2.create_instances(
    ImageId=ssm_response['Parameter']['Value'],
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName=key_name,
    SubnetId=subnet_1.id,
    SecurityGroupIds=[security_group_https.id],
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'OnPrem-Instance'
                },
            ]
        },
    ]
)

In [31]:
instance_id = ssm.describe_instance_information()['InstanceInformationList'][0]['InstanceId']
print("En la terminal ejecute el siguiente comando:")
print("   aws ssm start-session --target " + instance_id)

En la terminal ejecute el siguiente comando:
   aws ssm start-session --target i-01b3365c922ddd1ea


In [ ]:
#!/bin/bash
sudo su
rpm --import https://repo.mysql.com/RPM-GPG-KEY-mysql-2022
rpm -Uvh https://repo.mysql.com/mysql80-community-release-el7-3.noarch.rpm
yum install -y mysql-community-server
systemctl enable mysqld
systemctl start mysqld
export DbMasterPassword=Password@123
export DbMasterUsername=dbuser
mysql -u root "-p$(grep -oP '(?<=root@localhost\: )\S+' /var/log/mysqld.log)" -e "ALTER USER 'root'@'localhost' IDENTIFIED BY '${DbMasterPassword}'" --connect-expired-password
mysql -u root "-p${DbMasterPassword}" -e "CREATE USER '${DbMasterUsername}' IDENTIFIED BY '${DbMasterPassword}'"
mysql -u root "-p${DbMasterPassword}" -e "GRANT ALL PRIVILEGES ON *.* TO '${DbMasterUsername}'"
mysql -u root "-p${DbMasterPassword}" -e "FLUSH PRIVILEGES"
cd /var/lib/mysql-files/
wget https://datasets.imdbws.com/name.basics.tsv.gz
wget https://datasets.imdbws.com/title.akas.tsv.gz
wwget https://datasets.imdbws.com/title.basics.tsv.gz
wwget https://datasets.imdbws.com/title.crew.tsv.gz
wwget https://datasets.imdbws.com/title.episode.tsv.gz
wwget https://datasets.imdbws.com/title.principals.tsv.gz
wwget https://datasets.imdbws.com/title.ratings.tsv.gz
wgzip -d *.gz
curl -O https://www.amazondynamodblabs.com/static/rdbms-migration/rdbms-migration.zip
unzip -q rdbms-migration.zip
chmod 775 *.*
mysql -u root "-p${DbMasterPassword}" -e "CREATE DATABASE imdb;"
mysql -u root "-p${DbMasterPassword}" -e "CREATE TABLE imdb.title_akas (titleId VARCHAR(200), ordering VARCHAR(200),title VARCHAR(1000), region VARCHAR(1000), language VARCHAR(1000), types VARCHAR(1000),attributes VARCHAR(1000),isOriginalTitle VARCHAR(5),primary key (titleId, ordering));"
mysql -u root "-p${DbMasterPassword}" -e "CREATE TABLE imdb.title_basics (tconst  VARCHAR(200), titleType  VARCHAR(1000),primaryTitle  VARCHAR(1000), originalTitle  VARCHAR(1000), isAdult  VARCHAR(1000), startYear  VARCHAR(1000),endYear  VARCHAR(1000),runtimeMinutes  VARCHAR(1000),genres  VARCHAR(1000),primary key (tconst));"
mysql -u root "-p${DbMasterPassword}" -e "CREATE TABLE imdb.title_crew (tconst  VARCHAR(200), directors  VARCHAR(1000),writers  VARCHAR(1000),primary key (tconst));"
mysql -u root "-p${DbMasterPassword}" -e "CREATE TABLE imdb.title_principals (tconst  VARCHAR(200), ordering  VARCHAR(200),nconst  VARCHAR(200), category  VARCHAR(1000), job  VARCHAR(1000), characters  VARCHAR(1000),primary key (tconst,ordering,nconst));"
mysql -u root "-p${DbMasterPassword}" -e "CREATE TABLE imdb.title_ratings (tconst  VARCHAR(200), averageRating float,numVotes  integer,primary key (tconst));"
mysql -u root "-p${DbMasterPassword}" -e "CREATE TABLE imdb.name_basics (nconst  VARCHAR(200), primaryName  VARCHAR(1000),birthYear  VARCHAR(1000), deathYear  VARCHAR(1000), primaryProfession  VARCHAR(1000), knownForTitles VARCHAR(1000),primary key (nconst));"
mysql -u root "-p${DbMasterPassword}" -e "LOAD DATA INFILE '/var/lib/mysql-files/title_ratings.tsv' IGNORE INTO TABLE imdb.title_ratings FIELDS TERMINATED BY '\t';"
mysql -u root "-p${DbMasterPassword}" -e "LOAD DATA INFILE '/var/lib/mysql-files/title_basics.tsv'  IGNORE INTO TABLE imdb.title_basics FIELDS TERMINATED BY '\t';"
mysql -u root "-p${DbMasterPassword}" -e "LOAD DATA INFILE '/var/lib/mysql-files/title_crew.tsv' IGNORE INTO TABLE imdb.title_crew FIELDS TERMINATED BY '\t';"
mysql -u root "-p${DbMasterPassword}" -e "LOAD DATA INFILE '/var/lib/mysql-files/title_principals.tsv' IGNORE INTO TABLE imdb.title_principals FIELDS TERMINATED BY '\t';"
mysql -u root "-p${DbMasterPassword}" -e "LOAD DATA INFILE '/var/lib/mysql-files/name_basics.tsv' IGNORE INTO TABLE imdb.name_basics FIELDS TERMINATED BY '\t';"
mysql -u root "-p${DbMasterPassword}" -e "LOAD DATA INFILE '/var/lib/mysql-files/title_akas.tsv' IGNORE INTO  TABLE imdb.title_akas FIELDS TERMINATED BY '\t';"

Luego ejecute:

>> sudo su
>> cd /var/lib/mysql-files/
>> ls -lrt
>> mysql -u dbuser -pPassword@123

>> use imdb;
>> show tables;

In [ ]:
CREATE VIEW imdb.movies AS\
    SELECT tp.tconst,\
           tp.ordering,\
           tp.nconst,\
           tp.category,\
           tp.job,\
           tp.characters,\
           tb.titleType,\
           tb.primaryTitle,\
           tb.originalTitle,\
           tb.isAdult,\
           tb.startYear,\
           tb.endYear,\
           tb.runtimeMinutes,\
           tb.genres,\
           nm.primaryName,\
           nm.birthYear,\
           nm.deathYear,\
           nm.primaryProfession,\
           tc.directors,\
           tc.writers\
    FROM imdb.title_principals tp\
    LEFT JOIN imdb.title_basics tb ON tp.tconst = tb.tconst\
    LEFT JOIN imdb.name_basics nm ON tp.nconst = nm.nconst\
    LEFT JOIN imdb.title_crew tc ON tc.tconst = tp.tconst;

A título ilustrativo, a continuación se muestra un diagrama lógico que representa la relación entre varias tablas de origen que alojan el conjunto de datos IMDb.

* La tabla `title_basics` contiene las películas publicadas en EE.UU. después del año 2000. tconst es una clave alfanumérica asignada de forma exclusiva a cada película.
* `title_akas` contiene las regiones, los idiomas y los títulos de las películas publicadas. Es una relación 1:many con la tabla title_basics.
* `title_ratings` tiene la clasificación de las películas y el recuento de votos. Para este ejercicio, podemos suponer que la información se actualiza con alta frecuencia tras el estreno de la película. Tiene una relación 1:1 con la tabla title_basics
* `title_principals` contiene información sobre el reparto y el equipo. Tiene una relación 1:many con la tabla title_basics.
* `title_crew` tiene información sobre el guionista y el director. Tiene una relación 1:1 con la tabla title_basics.
* `name_basics` tiene información sobre el reparto y el equipo. Cada miembro tiene asignado un valor nconst único.

![Image](https://amazon-dynamodb-labs.com/images/migration31.jpg)

Utilice el siguiente comando para revisar el recuento de registros de la vista desnormalizada. En este punto, la base de datos de origen está lista para la migración a Amazon DynamoDB:

>> select count(*) from imdb.movies;